In [17]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.preprocessing import LabelEncoder 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [18]:
#Importamos los set de datos.
eventos = pd.read_csv("../events_up_to_01062018.csv", low_memory=False)
labels = pd.read_csv("../labels_training_set.csv")

In [19]:
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [20]:
#Creamos un dataframe de features
features = labels

In [21]:
features.shape

(19414, 2)

In [22]:
#Feature: Es un usuario que retorno alguna vez?

df = eventos.groupby(['person'])['new_vs_returning'].agg('count').reset_index()
df['retornoAlSitio'] = df["new_vs_returning"].apply(lambda x: True if x>0 else False)
del df['new_vs_returning']
features = pd.merge(features, df, on='person', how= 'left')


In [23]:
#Feature: Cantidad de tipo de estado de dispositivo que busco.

eventos_visualizacion = eventos.loc[eventos.event == 'viewed product']
df = eventos_visualizacion.groupby('person')['condition'].value_counts().unstack()
df = df.rename(columns={'Bom': 'cantidadVisualizacionesBuenEstado', 
                 'Bom - Sem Touch ID': 'cantidadVisualizacionesBuenEstadoSinTouch', 
                 'Novo': 'cantidadVisualizacionNuevo',
                 'Muito Bom': 'cantidadVisualizacionesMuyBuenEstado',
                 'Excelente': 'cantidadVisualizacionesExcelente'})

df = df.fillna(0)
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')

In [24]:
#Feature: Cantidad de eventos generados totales

df = eventos.groupby(['person'])['event'].agg('count').to_frame('cantidad_eventos_generados')
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')



In [25]:
#Features: Cantidad por tipo de evento generado
df = eventos.groupby('person')['event'].value_counts().unstack()
df = df.fillna(0)
df = df.rename(columns={'ad campaign hit': 'cantidadIngresosPorCampania', 
                 'brand listing': 'cantidadDeVisualizacionesPorMarca', 
                 'checkout': 'cantidadDeCheckouts',
                 'conversion': 'cantidadConversiones',
                 'generic listing': 'cantidadDeVisualizacionesDeLaHome',
                 'lead': 'cantidadDeGeneracionDeNotificaciones',
                 'search engine hit': 'cantidadDeIngresosPorBuscador',
                 'searched products': 'cantidadDeProductosBuscados',
                 'staticpage': 'cantidadDeVisitas',
                 'viewed product': 'cantidadDeVisualizacionesDeProducto',
                 'visited site': 'cantidadDeIngresosPorUrl',
                })
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])

features = pd.merge(features, df, on='person', how= 'left')

In [26]:
#Features: Cantidad de visualizaciones por marca
eventos_con_marca = eventos.loc[eventos.event == 'viewed product']
eventos_con_marca['marca'] = eventos_con_marca['model'].str.split(' ').str[0]
df = eventos_con_marca.groupby('person')['marca'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['cantidadDeVisualizacionesPor' + str(col) for col in df.columns]
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])
features = pd.merge(features, df, on='person', how= 'left')

C:\Users\felip\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
#Features: Realizo algun ingreso por tipo de canal proveniente generado
df = eventos.groupby('person')['channel'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['realizoIngresoPorCanal' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [28]:
#Features: Realizo algun ingreso por tipo de campania proveniente  generado
df = eventos.groupby('person')['campaign_source'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['realizoIngresoPorCamapania' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [29]:
#Features: Si alguna vez realizo una conversion

eventos_conversion = eventos.loc[eventos.event == 'conversion',]
df = eventos_conversion.groupby(['person'])['event'].agg('count').reset_index()
df['realizoConversion'] = df['event'].apply(lambda x: True if x>0 else False)
del df['event']
aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [30]:
#Features: Si alguna vez realizo un checkout

eventos_conversion = eventos.loc[eventos.event == 'checkout',]
df = eventos_conversion.groupby(['person'])['event'].agg('count').reset_index()
df['realizoCheckout'] = df['event'].apply(lambda x: True if x>0 else False)
del df['event']
aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [31]:
#Feature: Realizo algun ingreso por tipo de dispositivo
df = eventos.loc[eventos.event == 'visited site']
df = df.groupby(['person'])['device_type'].value_counts().unstack()

df.columns = ['realizoIngresoPorTipoDeDispositivo' + str(col) for col in df.columns]

for column in df:
    df[column] = df[column].apply(lambda x: True if x>0 else False)
    
aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)


features = pd.merge(features, aux, on='person', how='left')

In [32]:
#Feature: Es un usuario de brazil?
df = eventos.loc[eventos.event == 'visited site']
df = df.groupby(['person'])['country'].value_counts().unstack()
df['esUnUsuarioDeBrasil'] = df['Brazil'].apply(lambda x: True if x>0 else False)
df = df.loc[:,'esUnUsuarioDeBrasil']
df = df.reset_index()
aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [40]:
#Feature: cantidad de busquedas de acuerdo al storage del dispositivo
df = eventos.loc[eventos.event == 'viewed product']
df = df.groupby(['person'])['storage'].value_counts().unstack()
df = df.fillna(0)
df.columns = ['cantidadDeVisualizacionesPorTipoAlmacenamientoDe' + str(col) for col in df.columns]
scaler = MinMaxScaler()
for column in df:
    df[[column]] = scaler.fit_transform(df[[column]])
    
df.head(10)
features = pd.merge(features, df, on='person', how= 'left')

In [33]:
#Feature a que marca realizo la conversion. Dado que la mayoria son iphone, samsung  motorola vamos a armar una columna booleana con esas 3 marcas
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos['model'].str.split(' ').str[0]
df = eventos_con_marca.loc[eventos.event == 'conversion']
df = df.groupby(['person'])['marca'].value_counts().unstack()
df['realizoConversionAIphone'] = df['iPhone'].apply(lambda x: True if x>0 else False)
df['realizoConversionASamsung'] = df['Samsung'].apply(lambda x: True if x>0 else False)
df['realizoConversionAMotorola'] = df['Motorola'].apply(lambda x: True if x>0 else False)

df = df.loc[:,['realizoConversionAIphone','realizoConversionASamsung', 'realizoConversionAMotorola']]
df = df.reset_index()

aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')



In [34]:
#Feature a que marca realizo la checkout. Dado que la mayoria son iphone, samsung  motorola vamos a armar una columna booleana con esas 3 marcas
eventos_con_marca = eventos
eventos_con_marca['marca'] = eventos['model'].str.split(' ').str[0]
df = eventos_con_marca.loc[eventos.event == 'checkout']
df = df.groupby(['person'])['marca'].value_counts().unstack()
df['realizoCheckoutAIphone'] = df['iPhone'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutASamsung'] = df['Samsung'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutAMotorola'] = df['Motorola'].apply(lambda x: True if x>0 else False)
df['realizoCheckoutASony'] = df['Sony'].apply(lambda x: True if x>0 else False)

df = df.loc[:,['realizoCheckoutAIphone','realizoCheckoutASamsung', 'realizoCheckoutAMotorola','realizoCheckoutASony']]
df = df.reset_index()

aux = pd.merge(labels, df, on='person', how= 'left')
del aux['label']
aux = aux.fillna(False)
features = pd.merge(features, aux, on='person', how='left')


In [35]:
#Compleamos los campos NA con 0
features = features.fillna(0)

In [36]:
list(features)

['person',
 'label',
 'retornoAlSitio',
 'cantidadVisualizacionesBuenEstado',
 'cantidadVisualizacionesBuenEstadoSinTouch',
 'cantidadVisualizacionesExcelente',
 'cantidadVisualizacionesMuyBuenEstado',
 'cantidadVisualizacionNuevo',
 'cantidad_eventos_generados',
 'cantidadIngresosPorCampania',
 'cantidadDeVisualizacionesPorMarca',
 'cantidadDeCheckouts',
 'cantidadConversiones',
 'cantidadDeVisualizacionesDeLaHome',
 'cantidadDeGeneracionDeNotificaciones',
 'cantidadDeIngresosPorBuscador',
 'cantidadDeProductosBuscados',
 'cantidadDeVisitas',
 'cantidadDeVisualizacionesDeProducto',
 'cantidadDeIngresosPorUrl',
 'cantidadDeVisualizacionesPorAsus',
 'cantidadDeVisualizacionesPorLG',
 'cantidadDeVisualizacionesPorLenovo',
 'cantidadDeVisualizacionesPorMotorola',
 'cantidadDeVisualizacionesPorQuantum',
 'cantidadDeVisualizacionesPorSamsung',
 'cantidadDeVisualizacionesPorSony',
 'cantidadDeVisualizacionesPoriPad',
 'cantidadDeVisualizacionesPoriPhone',
 'realizoIngresoPorCanalDirect',
 'r

In [37]:
features.to_csv("../set_a_entrenar-definitivo.csv")

In [38]:
features.sample(30)

,person,label,retornoAlSitio,cantidadVisualizacionesBuenEstado,cantidadVisualizacionesBuenEstadoSinTouch,cantidadVisualizacionesExcelente,cantidadVisualizacionesMuyBuenEstado,cantidadVisualizacionNuevo,cantidad_eventos_generados,cantidadIngresosPorCampania,...,realizoIngresoPorTipoDeDispositivoTablet,realizoIngresoPorTipoDeDispositivoUnknown,esUnUsuarioDeBrasil,realizoConversionAIphone,realizoConversionASamsung,realizoConversionAMotorola,realizoCheckoutAIphone,realizoCheckoutASamsung,realizoCheckoutAMotorola,realizoCheckoutASony
5883,81edf57b,0,True,0.000000,0.000000,0.000000,0.000955,0.000000,0.000676,0.001912,...,False,False,True,False,False,False,True,False,False,False
2734,911f6bf3,0,False,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,False,False,False,False,False,False,False,True,False,False
18743,27771a92,0,True,0.008065,0.000000,0.000000,0.000000,0.000000,0.008114,0.015296,...,False,False,True,False,False,False,False,True,False,False
9296,e85297af,0,True,0.070565,0.072581,0.054028,0.034384,0.018182,0.066036,0.028681,...,False,False,True,True,False,False,True,False,False,False
4979,58f21eaf,1,True,0.002016,0.000000,0.000000,0.000955,0.000000,0.003381,0.007648,...,False,False,True,False,False,False,False,False,False,False
7258,5537640f,0,True,0.043347,0.000000,0.055924,0.019102,0.018182,0.038990,0.009560,...,False,False,True,False,False,False,False,True,False,False
10548,1f95ac0b,0,True,0.001008,0.000000,0.012322,0.005731,0.000000,0.005409,0.001912,...,False,False,True,False,False,False,True,False,False,False
1195,6c0d1e95,0,True,0.005040,0.000000,0.000948,0.001910,0.000000,0.009691,0.026769,...,False,False,True,False,False,False,False,True,False,False
9974,cd6082bd,0,True,0.000000,0.000000,0.000000,0.002865,0.000000,0.000902,0.001912,...,False,False,False,False,False,False,False,False,False,False
472,86571e0c,0,True,0.002016,0.000000,0.005687,0.001910,0.000000,0.005184,0.001912,...,False,False,True,False,False,False,False,True,False,False
